# EPH - Individuos


## Ajuste de ruta del proyecto en Notebook

In [1]:
import sys
import csv
from pathlib import Path

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

#CONSTANTS
from src.utils.constants import PROJECT_ROOT, DATA_SOURCE_DIR,DATA_CLEAN_DIR,FILENAME_INDIVIDUOS_UNIFIED
print(PROJECT_ROOT)

C:\Users\MONSO\code


## Unificación de los archivos de Individuos

In [2]:
from src.utils.helpers import  process_file, save_to_txt

# Procesar y guardar individuos
encabezados,individual = process_file(DATA_SOURCE_DIR, category="individual")
save_to_txt(encabezados,individual, DATA_CLEAN_DIR , FILENAME_INDIVIDUOS_UNIFIED)


✅ Archivo TXT guardado en: C:\Users\MONSO\code\data\clean\individuos_unificado.txt


## Procesamiento de los archivos

Abro el archivo unificado

In [3]:
from src.utils.constants import INDIVIDUOS_UNIFIED_DIR
from src.utils.helpers import read_file_dic
import csv

header,data=read_file_dic(INDIVIDUOS_UNIFIED_DIR)

Proceso el archivo

In [4]:
import src.procesamientos.individuos as indiv

# Agrego las nuevas columnas al header
header.extend(["CH04_str","NIVEL_ED_str","CONDICION_LABORAL","UNIVERSITARIO"])

# Proceso las nuevas columnas por cada fila
for row in data:
    indiv.add_data_ch04str(row)
    indiv.add_data_nivel_ed_str(row)
    indiv.add_data_cond_lab(row)
    indiv.add_data_universitario(row)

Guardo los datos procesados a un archivo

In [5]:
from src.utils.constants import INDIVIDUOS_PROCESSED_DIR
from src.utils.helpers import save_to_csv

save_to_csv(INDIVIDUOS_PROCESSED_DIR,header,data, delimiter=";")

## Análisis de Individuos

Abro el archivo de individuos procesado:

In [6]:
header,data=read_file_dic(INDIVIDUOS_PROCESSED_DIR)

Importo las funciones para realizar las consultas

In [7]:
import src.consultas.consultas as cs

### 1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.

In [8]:
cs.cant_personas_alfabetizadas(data)

Año	Alfabetizados	No alfabetizados
2024	13004244		216478


### 2- A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior

In [9]:
#Ingreso de datos por el usuario
ano=input("Ingrese el año: ")
trim=input("Ingrese el trimestre [1 a 4]: ")

cs.porc_extranjero_universitario(ano,trim,data)

El % de personas extranjeras que han cursado el nivel superior o universitario en el trimestre 2024 del año  es del: 0.00%


### 3- A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

In [10]:
cs.info_menor_desocupacion(data)

Valor mínimo de desocupación: 994334 en los siguientes años y trimestres:
Año: 2024, Trimestre: 3


### 4- Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más ocupantes con estudios universitarios o superiores finalizados. Información obtenida a partir del par de archivos más recientes. 

In [11]:
# 1. Cuento cuántos universitarios hay por hogar y guardar pondera

universitarios_por_hogar, pondera_por_hogar = indiv.contar_universitarios_y_pondera_por_hogar(data)

# 2. Filtro hogares con al menos 2 universitarios

hogares_filtrados = indiv.filtrar_hogares_con_min_universitarios(universitarios_por_hogar,pondera_por_hogar,min_universitarios=2)

In [12]:
# 3. Suma ponderada de todos los hogares

total_hogares = indiv.contar_hogares(pondera_por_hogar)

# 4. Suma poderada de hogares con 2+ universitarios por aglomerado

hogares_con_universitarios_min = indiv.contar_hogares(hogares_filtrados)

In [13]:
# 5. Calculo porcentajes  y ordeno por porcentajes
# También se imprime el nombre del aglomerado, el total de hogares y el total de hogares con al 
# menos 2 universitarios

top_5_aglomerados =indiv.obtener_top_n_porcentaje_hogares_universitarios(total_hogares, hogares_con_universitarios_min )

Procesando aglomerado: 32, Total: 2553606.0, Universitarios: 325386.0
Procesando aglomerado: 33, Total: 8325060.0, Universitarios: 316519.0
Procesando aglomerado: 22, Total: 136860.0, Universitarios: 4894.0
Procesando aglomerado: 19, Total: 222230.0, Universitarios: 5684.0
Procesando aglomerado: 13, Total: 1213389.0, Universitarios: 49044.0
Procesando aglomerado: 36, Total: 145361.0, Universitarios: 8218.0
Procesando aglomerado: 14, Total: 103390.0, Universitarios: 1769.0
Procesando aglomerado: 15, Total: 167588.0, Universitarios: 4591.0
Procesando aglomerado: 8, Total: 286244.0, Universitarios: 10044.0
Procesando aglomerado: 9, Total: 169476.0, Universitarios: 4642.0
Procesando aglomerado: 25, Total: 142664.0, Universitarios: 3021.0
Procesando aglomerado: 91, Total: 114479.0, Universitarios: 2552.0
Procesando aglomerado: 93, Total: 64497.0, Universitarios: 1877.0
Procesando aglomerado: 34, Total: 517778.0, Universitarios: 21264.0
Procesando aglomerado: 38, Total: 143868.0, Universitar

In [ ]:
# 7. Impresión del ranking de 5 aglomerados con mayor porcentaje de hogares con 2 o más universitarios
cs.imprimir_ranking_aglomerados(top_5_aglomerados, cantidad=5)

Ranking de los 5 aglomerados con mayor porcentaje de hogares con 2 o más universitarios:
1. Aglomerado 32 - Ciudad Autónoma de Buenos Aires: 12.74%
2. Aglomerado 2 - Gran La Plata: 7.72%
3. Aglomerado 6 - Gran Paraná: 6.71%
4. Aglomerado 10 - Gran Mendoza: 6.51%
5. Aglomerado 17 - Neuquén - Plottier: 6.23%


### 7- Informar para cada aglomerado el porcentaje de personas mayores que hayan cursado al menos en nivel universitario o superior.

Consideramos como muestra total para el cálculo porcentual sólo a las personas mayores de edad, que son las que tienen la posibilidad real de poder alcanzarlo.


In [15]:
cs.info_porcentual_educacionsuperior_aglomerado(data)

Aglomerado                               Porcentaje (%)
-------------------------------------------------------
32 - Ciudad Autónoma de Buenos Aires              63.64%
2 - Gran La Plata                                 44.70%
13 - Gran Córdoba                                 41.98%
3 - Bahía Blanca - Cerri                          41.59%
6 - Gran Paraná                                   40.92%
36 - Río Cuarto                                   40.39%
12 - Corrientes                                   39.82%
34 - Mar del Plata                                39.28%
4 - Gran Rosario                                  38.90%
10 - Gran Mendoza                                 38.64%
23 - Gran Salta                                   37.93%
19 - Jujuy - Palpalá                              37.26%
7 - Posadas                                       37.26%
29 - Gran Tucumán - Tafí Viejo                    36.64%
30 - Santa Rosa - Toay                            34.90%
17 - Neuquén - Plottier          

### 9- Pedir al usuario que seleccione un aglomerado y a partir de la información contenida retornar una tabla que contenga la cantidad de personas mayores de edad según su nivel de estudios alcanzados.


In [16]:
aglomerado = input("Ingrese el aglomerado: ")

cs.tabla_nivel_educativo(data, aglomerado)

Aglomerado     32 - Ciudad Autónoma de Buenos Aires
**************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
Año     Trimestre   Primario incompleto / Ed. especial      Primario completo                       Secundario incompleto                   Secundario completo                     Superior universitario incompleto       Superior universitario completo         Sin instrucción                         
********************************************************************************************************************************************************************************************************************************************************************************************************